
# Basic Data Manipulation and Analysis with `pandas`

Pandas was built on the premise of being a very good data handling and analysis tool through Python. According to their [documentation](http://docs.scipy.org/doc/scipy/reference/integrate.html) `pandas` **is an open source, BSD-licensed library providing high-performance, easy-to-use data structures and data analysis tools' for a Python environment**

In ECON 226 we are taught how to obtain large datasets of national surveys like the Canadian GSS Survey and to manipulate the data in excel to learn something meaningful from the variables of interests. What is more interesting was that all of the statistical tools were also taught in STAT 200, which used R for most of the analysis. The goal of this `pandas` guide is to take the statistical theory and apply it to a data set, which we will use `pandas` to manipulate instead of using excel or R.

## Goals for this Guide
* Explore how data can be **imported** into Jupyther and operated on by Pandas. File formats and file preparations. Including how to fix column datas that may be improperly imported.

* Discuss methods of data **filtration** and selection. Sometimes survey contains non responses categorized as certain numbers. So for statisticcal analysis certain datapoints will have to be removed first.

* Go through basic statistical functions and **statistical summaries** (ie. finding means standard deviations, IQR etc.) and manually calculate them too with `pandas` which will require **reorganizaion** of data as well.

* **Graphing** our data sets to visualize it will be most helpful. `pandas` is integrated with `matplotlib` so we will be able to use similar commands here too.

## Table of Contents

* #### Importing and Initially Assessing Data
	* Reading Data
	* Understanding the Data
    * Recognizing Import Flaws and Column Selection for Analysis
	* Slicing Rows 
* #### Preparing & Isolating Data For Analysis
    * Isolating Variables
	* Boolean Indexing and Filtering Data
	* Combining  Data Selection Criterions
	* Creating New Columns and Column Operations  
* #### Basic Data Analysis
	* Response Frequency
	* Counting Observations
	* Measures of Central Tendency
		* Mean
        * Weighted Mean
		* Median
		* Mode
	* Measures of Variability
		* Range
		* Standard Deviation
		* Percentiles and Quartiles
* #### Graphing and Visualizing Data
	* Histograms
	* Boxplot
	* Scatter Plots
* #### Exporting Data

## Dependencies

For `pandas` to work effectively, the following additional packages should be imported. Begin as such for all work with the `pandas` package.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Importing and Initially Assessing Data

In working in any statistical related fields, may it be Economics, you will be recieving data in one form another. If you are lucky, you may recieve it as a *comma sepparated* file type known as a `.csv` file or even a normal Microsoft Office Excel file `.xlxs`. Both of which `pandas` can read and write onto. For this guide, we will be using data from the Canadian Government's Cycle 25 General Social Survey (GSS) of 2011 which includes family related variables and is very large in size. Perfect for demonstrating the power of `pandas`

### Reading Data

To read data use the code `pd.read_csv(...)` where ... is the 'filename' with extension. For excel it is `pd.read_excel(...)`. Make sure to upload the datafile in the same directory as your Jupyter notebook for easy access

In our example for this guide, we will "read" the GSS data from a csv file and assign it to a variable name. In this case we use `dataset`. This will create what `pandas` regard as the **dataframe** . We will call our initial frame `gss`

In [ ]:
# to import 
gss = pd.read_csv('originalgss.csv')

# to view
gss

### Understanding the Data
As you can see above, each row is an "observation" with an attached number. They are survey responses an individual answered to a specific questions, starting from row zero. Each column is a variable and their variable name is in the row before row 0. 

The dataset does come with a PDF guide explaining what each of the variable is with descriptions for what the numerical value answers (Response codes) mean. For example:

![EDU5](http://i.imgur.com/lmsJBj7.png)

We can see that in regards to variable `EDU5` the person in row 0 with `RECID` (record ID number) 3221 has  a Doctorate/masters/bachelor's degree since the response code is a `1`

However, the response codes goes from 1 to 5 and then jumps to 8. It is a convention that after a skip in numerical codes the remaining possible codes reflect codes for non-responses or missing data which should not be used for data analysis.

### Recognizing Import Flaws and Column Selection for Analysis
Once the data is imported you should always keep an eyeout for data flaws. In this case obersve that the last column on the right (you may need to scroll that way), there is a column called `Unnamed: 9` with the same `NaN` response for everybody. This is clearly a useless column of data and should be removed from the data set.

We can do this by selecting the columns using their column names:

`dataset_name` followed by square brackets `[...]` where `...` is a list of variable names from the top of each column in the dataset that you want to include. `[...]` can be thought of as a "selection criteria" and in this case the list of column names is a selection criteria

In this example we created a new dataset from the original import dataset and called it `fix_gss`. Noticed that I did not include the name of the last column. Therefore it is no longer be in the newly created dataset and we now have a proper dataset to operate on.

In [ ]:
fix_gss = gss[["RECID", "WGHT_PER", "AGEC", "SEX", "HSDSIZEC", "FR_Q140", "EDU5", "SRH_Q110", "INCMC"]]
fix_gss

### Slicing Rows

You can also "slice" or remove rows in the the familar sence of selecting data fron an np.array by Selecting via [] with colons `:`

`dataset_name` followed by square brackets `[A:B]` where `A` is the first row and `B` is the last desired row. Remember that things are indexed from 0.

In [ ]:
# Here we select row 0 to row 3 you can do this via [0:3] or [:3]
fix_gss[:3] 

## Preparing & Isolating Data For Analysis

### Isolating Variables
Lets prepare the data now so that we can examine the education level of Canadians. For this we should select a dataframe of only relevant variables: `["RECID",'WGHT_PER','EDU5']` by creating a smaller dataset with only these variables from `fix_gss` call it `EDU5_sheet`. Note that `WGHT_PER` refers to how many people of the population this respodnent represent. For example, person ID 3221 represents 710.9... people in the population of Canada and keep in mind that people can not be reprsented as a decimal, but we will adress this later when we consider weights again.

In [ ]:
EDU5_sheet = fix_gss[["RECID",'WGHT_PER','EDU5']]

EDU5_sheet[:3] # just to show the first 3 entries.

### Boolean Indexing and Filtering Data

Recall that the response codes of the variable EDU5 goes from 1 to 5 and then jumps to 8/9. This aims to sepparate meaning ful data from non-response data. For analysis we should always remove these non-response data. To do so we will use "boolean indexing" based on a single column’s values.

The full command is as follows:

`EDU5_sheet_filt = fix_gss[cleaned_data.EDU5 < 7]`

* **EDU5_filt** is the new filtered dataset name
* **cleaned_data** is the call to the dataset we want to operate on.
* **[...]** can be though as a data selection criteria 
* **fix_gss.EDU5** is isolating the column `EDU5` in the `fix_gss` dataset that we want `pandas` to run the question
* **fix_gss.EDU5 < 7** is the question: select data rows where the column with name `EDU5` has value `<7`

Notice that the result only has 22262 rows/respondents (indicated at the bottom of the resulting table). This can be verified as 22,435 - (83+90) = 22,262. Where 83 is the number of "Not Stated" responses and "90" are the numbert of "Don't Know" responmses. Both count as non responses that needs to be filtered out before analysis.

In [ ]:
EDU5_sheet_filt = fix_gss[fix_gss.EDU5 < 7]
EDU5_sheet_filt

### Combining Data Selection Criterions

Let's say you want to select only a few columns of data from a dataset and immediately filter out the non responses, you can do that in one variable definition rather than subdivinding the data once to isolate the columns and then again to filter the data and so forth by simply writting the criterians next to each other as such

`dataset_name[criterian 1][criterian 2]...[criterian N]`

Let's properly isolate and filter the data for EDU5 efficiently as such:

In [ ]:
EDU5_sheet = fix_gss[["RECID",'WGHT_PER','EDU5']][fix_gss.EDU5 < 7]
EDU5_sheet[:3]

### Creating New Columns and Column Operations

Sometimes you will want to create a new solumn of data with data that already exists. For example. in this `EDU5_sheet` data set, we can see that the weights are not representative of whole people since they have decimals. So we would like to some how round each value, maybe with the Python function `round(...)`

To do so we use the code:
EDU5_sheet['WEIGHT'] = round(EDU5_sheet['WGHT_PER'])

* `EDU_sheet` calls the desured dataset
* `['WEIGHT']` this selection criterion actually creates a new column so that by commanding `EDU5_sheet['WEIGHT']` will bring up the desired values.
* `round(EDU5_sheet['WGHT_PER'])` tells pandas to round every datapoint in the `WGHT_PER` coluimn of the `EDU5_sheet` dataset.


In [ ]:
EDU5_sheet['WEIGHT'] = round(EDU5_sheet['WGHT_PER'])
EDU5_sheet[:3]

You can rearrange the columns with the skills you have learned in selecting columns. Lets move the new `WEIGHTS` column next to the column `WGHT_PER`

In [ ]:
EDU5_sheet = EDU5_sheet[["RECID", "WGHT_PER", "WEIGHT", "EDU5"]]
# in the selection criterion we just lists the columns in the order we want
EDU5_sheet[:3]

## Basic Data Analysis

When you want `pandas` to do something or find something on a dataset you will usually do so by appending a `.operation` to a dataset

### Response Frequency

Lets say we want to find the frequency of each responses:

call the `dataset_name` then square brackets to set a data criterian for an operation `[...]` where the `...` is the name of the column or the variable name. Followed by the operation `.value_counts()`

This should match the FREQ noted in the GSS guide, which they do.

In [ ]:
EDU5_sheet['EDU5'].value_counts()

You can also set this as a new dataset if you want as such:

In [ ]:
EDU5_freq = EDU5_sheet['EDU5'].value_counts()
EDU5_freq

### Counting Observations
Sometimes we will need to count how many non null (blank) observations there are. We will use the operator `.count()`

In [ ]:
EDU5_sheet['EDU5'].count()

### Measures of Central Tendency
Understanding the "centers" of data and their properties

#### Mean

The mean or average $\bar{x}$ is the most widely used measurement of central tendancy. It is calculated by adding up the numbers ($x$) in the dataset and then dividing by the number of $n$ observations.
$$
\bar{x} = \frac{\sum x}{n}
$$

To calculated the mean in `pandas` this is the code structure:

`EDU5_sheet['EDU5'].mean()`
* `EDU5_sheet` is the name of the data set
* `['EDU5']` is the selection you want to operate on
* `.mean()` is the operation

In [ ]:
EDU5_sheet['EDU5'].mean()

#### Weighted Mean

Now we will calculated mean which will give us the real estimate of the population mean rather than simply the sample mean which is what we calculated before based on the gss data. 

The weighted mean is calculated as follows for a given set of obersvation values ${x_1,x_2,...,x_n}$ and $w_n$ are the correspomding weights to each obeservation:

$$
\bar{x} = \frac{\sum_{i=1}^n w_ix_i}{\sum_{i=1}^n w_i} = \frac{w_1x_1 + w_2x_2 +  \cdots + w_nx_n}{w_1 + w_2 +  \cdots + w_n}
$$

To do so we should create the terms in the numerator as a new column. The result of which is the multiplicationn of the rounded weight and the EDU5 observation value.

To do this we will use the code:

`EDU5_sheet['WGHT_EDU5'] = EDU5_sheet['EDU5'] * EDU5_sheet['WEIGHT']`

* `EDU5_sheet['WGHT_EDU5']` calls up the dataset we want to operate on and sets the name of the new column through `['WGHT_EDU5']`
* we equate this to the multiplication of the `'EDU5'` `'WEIGHT'` column
* which we call up to be operated through its name and selection criterion as such `EDU5_sheet['EDU5']`
* the multiplcation sign `*` tells pandas we want to multiply the data with something else, in this case data from `EDU5_sheet['WEIGHT']`, which is the weight column.

In [ ]:
EDU5_sheet['WGHT_EDU5'] = EDU5_sheet['EDU5'] * EDU5_sheet['WEIGHT']
EDU5_sheet

No we will find the summation of the numerator of the above equation with the `.sum` operator similar to what we would do in summing lists or arrays.

So we will call up the new column `'WGHT_EDU5'` and then appending the `.sum` opertor giving us the code `EDU5_sheet['WGHT_EDU5'].sum()` we will set this equal to `WGHT_numr`

Then we will call up the `WEIGHT` and summing it similarly to find the denominator of the weighted mean formula and setting it to be equal to `WGHT_denom` by using the code `EDU5_sheet['WEIGHT'].sum()`

Then the weighted average is equal to `WGHT_numr / WGHT_denom`

In [ ]:
WGHT_numr = EDU5_sheet['WGHT_EDU5'].sum()
WGHT_denom = EDU5_sheet['WEIGHT'].sum()

weighted_avg = WGHT_numr / WGHT_denom
weighted_avg

Since EDU5 is a **ranked qualitative data** the calculated mean and weighted mean actually means something. In this case it tells that on everage canadians has an education level of 2.75 on the GSS scale, which means that Canadians, on average, tends to have some sort of university/college degree. If the qualitative data is **unranked** it will be meaningless and additional treatments would be necessary to make it more meaningful. **Quantitative data** is always statistically meaningful.

#### Median

The median is the middle value (if there is a unique middle value), or the average of the two middle values (when the data set has even observations), in an ordered data set.

`pandas` can calculate the median with the operator `.median`

Let's find out then the median of `EDU5` respondents

In [ ]:
median = EDU5_sheet['EDU5'].median()
median

This can be interpreted that at least 50% of the population has a graduate degree (1) or diploma of some sort (2) 

#### Mode
The mode is the most frequently occuring value in the dataset. `pandas` can calculate the median with the operator `.mode`. Let's find out then the mode of `EDU5` respondents.

`pandas` will always return the mode in two column. The left column will signify the first mode (index zero). For examplem if the data set is ${1, 1, 3, 3, 3, 5, 5, 7, 7, 7}$ `pandas` will return

$$
\begin{array}{cc}
\texttt{0} & \texttt{3}\\
\texttt{1} & \texttt{7} \end{array}
$$

Where there are two modes, 3 and 7, since they both appear 3 times. 3 is the first, therefore 0-th mode and 7 is the second (1-st mode)

In [ ]:
mode = EDU5_sheet['EDU5'].mode()
mode

### Measures of Variability

In statistics, we are also interested in how variable the data set is. Or how different they are from each other and the mean.

#### Range

The range of a data set is the difference between the smallest and the largest obserbed value.

Since there is no `.range` function, we will use `pandas` to find the `.max()` and the `.min` of a dataset respectively to find the difference. 

In [ ]:
therange = EDU5_sheet['EDU5'].max() - EDU5_sheet['EDU5'].min()
therange

#### Standard Deviation

Though the range can be useful in some cases, it does not distinguish differences between datasets of the same max and min values but very different variability. The better measure of variability is the standard deviation, based on measuring the deviations from the mean and is represented by the signma $\sigma$ symbol. 

The standard deviation is equal to the squareroot of the sum of the  squares of the difference between an observation and the mean $\mu$, divided by the number of observations $N$

$$
\sigma = \sqrt{\frac{\sum(x-\mu)^2}{N}}
$$

We will first demonstrate how to do this manually which will go over most of the skills learnt thus far, and then by using the formula  `std`.

For this excercise we will use the original data `fix_gss` and `"HSDSIZEC"` (household size) data

In [ ]:
# step 1: find the mean
sample_mean = fix_gss['HSDSIZEC'].mean()

# step 2: find the square of differences between observation and mean
# we will need to create a new column of data for this call it `hsd_dif_sqr`

fix_gss['hsd_dif_sqr'] = (fix_gss['HSDSIZEC'] - sample_mean)**2
fix_gss[:3]

In [ ]:
# step 3: count the sample size
N = fix_gss['HSDSIZEC'].count()

# step 4: find the sum
sum_of_difference2 = fix_gss['hsd_dif_sqr'].sum()
sum_of_difference2
# step 5: use the s.d formula
sd = np.sqrt(sum_of_difference2/N)
sd

Let's verify this data now with the `.std()` operator

In [ ]:
fix_gss['HSDSIZEC'].std()

Not bad!

#### Percentiles and Quartiles

While the standard deviation is very fundamental to statistical analysis of variance, it can significantly be affected by outliers. Percentiles/Quartiles are not affected by outliers. The p-th percentile ie the value at which p% of observations lies below. The first quartile is the 25%th percentile where it is the datapoint at which 25% of all data points lie below. Notice that the 50%th percentile is the same definition as the median. We will use this for verification.

`pandas` can find the percentiles easily with the `.quantile(p)` operator. Where p is the desired percentile in decimals (ie. 25%th percentile is 0.25). You can also give the operator a list of percentiles to compute.

Let's calculated the median two ways, first with the `.median()` operator and second with the `quantile(0.5)` operator. 

In [ ]:
fix_gss['HSDSIZEC'].median()

In [ ]:
fix_gss['HSDSIZEC'].quantile(0.5)

Yay! A match! . Lets compute a random quantile and all the interquartile ranges as a list `[0.25,0.5,0.75]`

In [ ]:
fix_gss['HSDSIZEC'].quantile(0.1)

In [ ]:
fix_gss['HSDSIZEC'].quantile([0.25,0.5,0.75])

We can also find the Interquartile range which is the 3rd quartile minus the 1st quartile

In [ ]:
IQR = fix_gss['HSDSIZEC'].quantile(0.75) - fix_gss['HSDSIZEC'].quantile(0.25)
IQR

## Graphing and Visualizing Data

Pandas dataframes will accept matplotlib commands to visualize the data at hand. The general formatting of the code begins with calling the data set and the column of data we want to visualize, followed by the type of diagram we want.

### Histogram
To make the histogram, which is a diagram which shows the frequency of observation of each possible answer the code format is as follows:

`fix_gss['HSDSIZEC'].hist().set_ylabel('Frequency')`

* First call up the dataset and the columns we want the plotting command to assess with `ix_gss['HSDSIZEC']`
* use the `.hist()` `matplotlib` command to make the histogram based of the called data
* use any `matplotlib` commands you like, such as `set_ylabel('Frequency')` to format the diagram

In [ ]:
fix_gss['HSDSIZEC'].hist().set_ylabel('Frequency')

### Box Plot

The box plot diagram is ideal for displaying quartiles including the median. In the diagram below we can see several elements:
* The top and bottom T's are the max and the min values of the dataset.
* The edges of the middle box is the first and third quartile value
* The green line in the middle of the box is the 2nd quartile or the median.

The command is `plot.box()` rather than the `.hist()` used in the above example.

In [ ]:
fix_gss['AGEC'].plot.box().set_ylabel('AGE')

## Scatter Plots

An instance where the original template of code is a bit different comes in the forms of scatter plots. Instead of calling the spefic columns before hand through something like `fix_gss['AGEC']` for calling the AGEC data, we only first call the dataset and call the columns within the scatter function. For example:

`fix_gss.plot.scatter(x='AGEC', y='HSDSIZEC', lw=0, alpha=0.01)`

* `fix_gss` calls the dataset to be operated on
* `.plot.scatter(...)` is the scatter plot operation.
* to set where the plot gets it's x and y values from, equate it to the column names sepparated by commas like `x='AGEC', y='HSDSIZEC'` 
* You may also follow it up with other `matplotlib` parameters such as, `alpha` density of each plot point or the `lw` line width.

In [ ]:
fix_gss.plot.scatter(x='AGEC', y='HSDSIZEC', lw=0, alpha=0.01)

## Exporting Data

In closing this guide, I would like to take the time to appreciate the different new datasets we have create for analysis and exporting them, which includes `fix_gss` and `EDU5_sheet`. We will now export them as CSVs with the code 

`fix_gss.to_csv('foo.csv')`

* where `fix_gss` is the call to the created dataset within our coding environment
* `.to_csv(...)` is the operation to export to csv.
* to export to an excel file, use instead the `.to_excel('foo.xlsx', sheet_name='Sheet1')` command, but notice that we have to set as well the sheet_name.

In [ ]:
fix_gss.to_csv('fixgss.csv')

![EXPORTED](http://i.imgur.com/zonJMRh.png)